In [81]:
import pandas as pd
import numpy as np

In [117]:
df = pd.read_csv('train.csv', encoding='utf-8-sig')
df_test = pd.read_csv('test.csv')

In [118]:
#신고일자
df['신고일자'] = pd.to_datetime(df['신고일자'])
df.insert(1, '신고일자_월', df['신고일자'].dt.month)
df = df.drop('신고일자', axis=1)

In [119]:
#A, C, CIT, E, F, FAS, FAU, FCA, FCL, FCN, FEF, FIN, FNZ, FPE,
#FSG, FTR, FUS, FVN, L, P, R, W
#22개

In [120]:
mapping = {'A':'A', 'C':'C', 'C3':'C', 'CIT':'CIT', 'CIT1':'CIT',
          'E1':'E', 'F':'F', 'FAS1':'FAS', 'FAU1':'FAU', 'FCA1':'FCA1',
          'FCL1':'FCL', 'FCN1':'FCN', 'FCN2':'FCN', 'FCN4':'FCN',
          'FCN6':'FCN', 'FEF1':'FEF', 'FEU1':'FEU', 'FEU6':'FEU',
          'FIN1':'FIN', 'FNZ1':'FNZ', 'FPE1':'FPE',  'FSG1':'FSG',
          'FTR1':'FTR', 'FUS1':'FUS', 'FUS2':'FUS', 'FUS5':'FUS',
          'FUS6':'FUS', 'FVN1':'FVN', 'FVN2':'FVN', 'L':'L', 'P1':'P',
          'P3':'P', 'R':'R', 'W1':'W', 'W2':'W'}

In [121]:
df['관세율구분코드'] = df['관세율구분코드'].map(mapping)
df['관세율구분코드'].sample(10)

21299    FUS
29243      C
1125       A
11927      A
65829    CIT
35432    FCN
64645      A
65152      C
31854      A
81853      A
Name: 관세율구분코드, dtype: object

In [122]:
df['관세율구분코드'].unique()

array(['A', 'FCN', 'C', 'FEU', 'CIT', 'FVN', 'FEF', 'FUS', 'F', 'FAU',
       'E', 'L', 'W', 'FCA1', 'FAS', 'P', nan, 'FIN', 'FCL', 'FTR', 'FSG',
       'R', 'FPE', 'FNZ'], dtype=object)

In [123]:
 # 항목 삭제
df = df.drop(['신고인부호', '특송업체부호','수입자부호','해외거래처부호'], axis=1)

In [124]:
# label encoding
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
label = df['수입통관계획코드']
le.fit(label)
label_encoded = le.transform(label)
l_e_df = pd.DataFrame(label_encoded, columns=['label_encoded'])
df['수입통관계획코드'] = l_e_df

label = df['수입신고구분코드']
le.fit(label)
label_encoded = le.transform(label)
l_e_df = pd.DataFrame(label_encoded, columns=['label_encoded'])
df['수입신고구분코드'] = l_e_df

le = LabelEncoder()
label = df['원산지국가코드']
le.fit(label)
label_encoded = le.transform(label)
l_e_df = pd.DataFrame(label_encoded, columns=['label_encoded'])
df['원산지국가코드'] = l_e_df

le = LabelEncoder()
label = df['적출국가코드']
le.fit(label)
label_encoded = le.transform(label)
l_e_df = pd.DataFrame(label_encoded, columns=['label_encoded'])
df['적출국가코드'] = l_e_df

df=df.drop(['관세율구분코드'],axis=1)

In [125]:
df.sample(3)

,신고번호,신고일자_월,통관지세관부호,수입통관계획코드,수입신고구분코드,수입거래구분코드,수입종류코드,징수형태코드,신고중량(KG),과세가격원화금액,운송수단유형코드,반입보세구역부호,HS10단위부호,적출국가코드,원산지국가코드,관세율,우범여부,핵심적발
56089,28064,7,20,2,1,87,21,11,4242.5,1.239856e+07,10,4002549,8529909910,31,46,0.0,0,0
44456,370117,6,40,2,1,11,21,11,3331.4,1.942547e+05,10,4077007,3506109000,81,95,0.0,0,0
46468,46515,6,40,3,1,11,21,11,287.6,2.050429e+06,40,4002001,8479905010,15,19,0.0,0,0


In [126]:
ca=['신고번호','신고일자_월','통관지세관부호','수입통관계획코드', '수입신고구분코드',
   '수입거래구분코드','수입종류코드','징수형태코드','신고중량(KG)','과세가격원화금액',
   '운송수단유형코드','반입보세구역부호','HS10단위부호','적출국가코드',
   '원산지국가코드','관세율','우범여부','핵심적발']

In [127]:
# 표준화 
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler() 
scaler.fit(df)
df_scaled = scaler.transform(df)

df_scaled = pd.DataFrame(data=df_scaled,columns=ca)

In [128]:
# 정규화
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() 
scaler.fit(df)
df_scaled = scaler.transform(df)

df_scaled = pd.DataFrame(data=df_scaled, columns=ca)

## XGBoost

In [129]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import plot_importance
from xgboost import XGBClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, confusion_matrix

In [130]:
def metrics(y_test, pred):
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    roc_score = roc_auc_score(y_test, pred)
    print('정확도 : {0:.2f}, 정밀도 : {1:.2f}, 재현율 : {2:.2f}'.format(accuracy, precision, recall))
    print('f1-score : {0:.2f}, auc : {1:.2f}'.format(f1, roc_score))

In [131]:
cx=['신고번호', '통관지세관부호', '수입통관계획코드', '수입신고구분코드',
                                      '수입거래구분코드','수입종류코드','징수형태코드','신고중량(KG)','과세가격원화금액',
                                      '운송수단유형코드','반입보세구역부호','HS10단위부호','적출국가코드',
  '원산지국가코드','관세율구분코드', '관세율']

In [183]:
# 우범여부
Xu = np.array(pd.DataFrame(df_scaled, columns=cx))
Y_u = np.array(pd.DataFrame(df_scaled, columns=['우범여부']))
Y_u = Y_u.astype('int')
evals=[(X_test, y_u_test)]
Xu_train, Xu_test, y_u_train, y_u_test = train_test_split(Xu, Y_u, test_size=0.2, random_state=11)
print(Xu_train.shape, Xu_test.shape)
evals=[(Xu_test, y_u_test)]

(71695, 16) (17924, 16)


In [184]:
# 데이터학습
xgb = XGBClassifier(n_estimators=500, learning_rate = 0.1, max_depth = 4)
df_xgb_u = xgb.fit(Xu_train, y_u_train,early_stopping_rounds=100,eval_metric="logloss", eval_set=evals,verbose=True)
xgb_pred = xgb.predict(Xu_test)

C:\Users\82108\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\82108\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


[0]	validation_0-logloss:0.65735
[1]	validation_0-logloss:0.62799
[2]	validation_0-logloss:0.60352
[3]	validation_0-logloss:0.58324
[4]	validation_0-logloss:0.56605
[5]	validation_0-logloss:0.55165
[6]	validation_0-logloss:0.53938
[7]	validation_0-logloss:0.52888
[8]	validation_0-logloss:0.52003
[9]	validation_0-logloss:0.51228
[10]	validation_0-logloss:0.50514
[11]	validation_0-logloss:0.49836
[12]	validation_0-logloss:0.49340
[13]	validation_0-logloss:0.48907
[14]	validation_0-logloss:0.48522
[15]	validation_0-logloss:0.48201
[16]	validation_0-logloss:0.47908
[17]	validation_0-logloss:0.47622
[18]	validation_0-logloss:0.47306
[19]	validation_0-logloss:0.47115
[20]	validation_0-logloss:0.46927
[21]	validation_0-logloss:0.46702
[22]	validation_0-logloss:0.46555
[23]	validation_0-logloss:0.46381
[24]	validation_0-logloss:0.46230
[25]	validation_0-logloss:0.46119
[26]	validation_0-logloss:0.45992
[27]	validation_0-logloss:0.45896
[28]	validation_0-logloss:0.45786
[29]	validation_0-loglos

[238]	validation_0-logloss:0.43440
[239]	validation_0-logloss:0.43440
[240]	validation_0-logloss:0.43441
[241]	validation_0-logloss:0.43442
[242]	validation_0-logloss:0.43441
[243]	validation_0-logloss:0.43443
[244]	validation_0-logloss:0.43443
[245]	validation_0-logloss:0.43442
[246]	validation_0-logloss:0.43445
[247]	validation_0-logloss:0.43445
[248]	validation_0-logloss:0.43446
[249]	validation_0-logloss:0.43448
[250]	validation_0-logloss:0.43443
[251]	validation_0-logloss:0.43440
[252]	validation_0-logloss:0.43437
[253]	validation_0-logloss:0.43438
[254]	validation_0-logloss:0.43437
[255]	validation_0-logloss:0.43433
[256]	validation_0-logloss:0.43436
[257]	validation_0-logloss:0.43436
[258]	validation_0-logloss:0.43438
[259]	validation_0-logloss:0.43436
[260]	validation_0-logloss:0.43437
[261]	validation_0-logloss:0.43435
[262]	validation_0-logloss:0.43433
[263]	validation_0-logloss:0.43430
[264]	validation_0-logloss:0.43434
[265]	validation_0-logloss:0.43434
[266]	validation_0-l

In [186]:
metrics(y_u_test, xgb_pred)

정확도 : 0.79, 정밀도 : 0.56, 재현율 : 0.22
f1-score : 0.32, auc : 0.59


In [198]:
# 핵심적발
cu=['신고번호', '통관지세관부호', '수입통관계획코드', '수입신고구분코드',
                                      '수입거래구분코드','수입종류코드','징수형태코드','신고중량(KG)','과세가격원화금액',
                                      '운송수단유형코드','반입보세구역부호','HS10단위부호','적출국가코드',
  '원산지국가코드','관세율구분코드', '관세율','우범여부']

Xh = np.array(pd.DataFrame(df_scaled, columns=cu))
Y_h = np.array(pd.DataFrame(df_scaled, columns=['핵심적발']))
Y_h = Y_h.astype('int')
Xh_train, Xh_test, y_h_train, y_h_test = train_test_split(Xh, Y_h, test_size=0.2, random_state=11)
print(Xh_train.shape, Xh_test.shape)
evals=[(Xh_test, y_h_test)]

(71695, 17) (17924, 17)


In [199]:
xgb = XGBClassifier(n_estimators=500, learning_rate = 0.1, max_depth = 4)
df_xgb_h = xgb.fit(Xh_train, y_h_train,early_stopping_rounds=100,eval_metric="logloss", eval_set=evals,verbose=True)
xgb_pred = xgb.predict(Xh_test)

[0]	validation_0-logloss:0.61870
[1]	validation_0-logloss:0.55773
[2]	validation_0-logloss:0.50688
[3]	validation_0-logloss:0.46390
[4]	validation_0-logloss:0.42719
[5]	validation_0-logloss:0.39558
[6]	validation_0-logloss:0.36813
[7]	validation_0-logloss:0.34417
[8]	validation_0-logloss:0.32324
[9]	validation_0-logloss:0.30479
[10]	validation_0-logloss:0.28853
[11]	validation_0-logloss:0.27411
[12]	validation_0-logloss:0.26132
[13]	validation_0-logloss:0.24991
[14]	validation_0-logloss:0.23978
[15]	validation_0-logloss:0.23072
[16]	validation_0-logloss:0.22263
[17]	validation_0-logloss:0.21539
[18]	validation_0-logloss:0.20890
[19]	validation_0-logloss:0.20308
[20]	validation_0-logloss:0.19783
[21]	validation_0-logloss:0.19317
[22]	validation_0-logloss:0.18896
[23]	validation_0-logloss:0.18518
[24]	validation_0-logloss:0.18177
[25]	validation_0-logloss:0.17868
[26]	validation_0-logloss:0.17589
[27]	validation_0-logloss:0.17342
[28]	validation_0-logloss:0.17117
[29]	validation_0-loglos

In [200]:
metrics(y_h_test, xgb_pred)

정확도 : 0.90, 정밀도 : 0.53, 재현율 : 0.68
f1-score : 0.60, auc : 0.80


## early_stopping_rounds 10으로 학습

In [217]:
# 우범여부
Xu = np.array(pd.DataFrame(df_scaled, columns=cx))
Y_u = np.array(pd.DataFrame(df_scaled, columns=['우범여부']))
Y_u = Y_u.astype('int')
evals=[(X_test, y_u_test)]
Xu_train, Xu_test, y_u_train, y_u_test = train_test_split(Xu, Y_u, test_size=0.2, random_state=11)
print(Xu_train.shape, Xu_test.shape)
evals=[(Xu_test, y_u_test)]

xgb = XGBClassifier(n_estimators=500, learning_rate = 0.1, max_depth = 4)
df_xgb_u = xgb.fit(Xu_train, y_u_train,early_stopping_rounds=10,eval_metric="logloss", eval_set=evals,verbose=True)
xgb_pred = xgb.predict(Xu_test)

metrics(y_u_test, xgb_pred)

(71695, 16) (17924, 16)
[0]	validation_0-logloss:0.65735


C:\Users\82108\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\82108\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


[1]	validation_0-logloss:0.62799
[2]	validation_0-logloss:0.60352
[3]	validation_0-logloss:0.58324
[4]	validation_0-logloss:0.56605
[5]	validation_0-logloss:0.55165
[6]	validation_0-logloss:0.53938
[7]	validation_0-logloss:0.52888
[8]	validation_0-logloss:0.52003
[9]	validation_0-logloss:0.51228
[10]	validation_0-logloss:0.50514
[11]	validation_0-logloss:0.49836
[12]	validation_0-logloss:0.49340
[13]	validation_0-logloss:0.48907
[14]	validation_0-logloss:0.48522
[15]	validation_0-logloss:0.48201
[16]	validation_0-logloss:0.47908
[17]	validation_0-logloss:0.47622
[18]	validation_0-logloss:0.47306
[19]	validation_0-logloss:0.47115
[20]	validation_0-logloss:0.46927
[21]	validation_0-logloss:0.46702
[22]	validation_0-logloss:0.46555
[23]	validation_0-logloss:0.46381
[24]	validation_0-logloss:0.46230
[25]	validation_0-logloss:0.46119
[26]	validation_0-logloss:0.45992
[27]	validation_0-logloss:0.45896
[28]	validation_0-logloss:0.45786
[29]	validation_0-logloss:0.45690
[30]	validation_0-loglo

In [216]:
# 핵심적발
cu=['신고번호', '통관지세관부호', '수입통관계획코드', '수입신고구분코드',
                                      '수입거래구분코드','수입종류코드','징수형태코드','신고중량(KG)','과세가격원화금액',
                                      '운송수단유형코드','반입보세구역부호','HS10단위부호','적출국가코드',
  '원산지국가코드','관세율구분코드', '관세율','우범여부']

Xh = np.array(pd.DataFrame(df_scaled, columns=cu))
Y_h = np.array(pd.DataFrame(df_scaled, columns=['핵심적발']))
Y_h = Y_h.astype('int')
Xh_train, Xh_test, y_h_train, y_h_test = train_test_split(Xh, Y_h, test_size=0.2, random_state=11)
print(Xh_train.shape, Xh_test.shape)
evals=[(Xh_test, y_h_test)]

xgb = XGBClassifier(n_estimators=500, learning_rate = 0.1, max_depth = 4)
df_xgb_h = xgb.fit(Xh_train, y_h_train,early_stopping_rounds=10,eval_metric="logloss", eval_set=evals,verbose=True)
xgb_pred = xgb.predict(Xh_test)

metrics(y_h_test, xgb_pred)

(71695, 17) (17924, 17)
[0]	validation_0-logloss:0.61870


C:\Users\82108\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\82108\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


[1]	validation_0-logloss:0.55773
[2]	validation_0-logloss:0.50688
[3]	validation_0-logloss:0.46390
[4]	validation_0-logloss:0.42719
[5]	validation_0-logloss:0.39558
[6]	validation_0-logloss:0.36813
[7]	validation_0-logloss:0.34417
[8]	validation_0-logloss:0.32324
[9]	validation_0-logloss:0.30479
[10]	validation_0-logloss:0.28853
[11]	validation_0-logloss:0.27411
[12]	validation_0-logloss:0.26132
[13]	validation_0-logloss:0.24991
[14]	validation_0-logloss:0.23978
[15]	validation_0-logloss:0.23072
[16]	validation_0-logloss:0.22263
[17]	validation_0-logloss:0.21539
[18]	validation_0-logloss:0.20890
[19]	validation_0-logloss:0.20308
[20]	validation_0-logloss:0.19783
[21]	validation_0-logloss:0.19317
[22]	validation_0-logloss:0.18896
[23]	validation_0-logloss:0.18518
[24]	validation_0-logloss:0.18177
[25]	validation_0-logloss:0.17868
[26]	validation_0-logloss:0.17589
[27]	validation_0-logloss:0.17342
[28]	validation_0-logloss:0.17117
[29]	validation_0-logloss:0.16913
[30]	validation_0-loglo

In [218]:
# test
t_df = pd.read_csv('test.csv')

#신고일자, 해외거래처부호, 특송업체부호 삭제
t_df = t_df.drop(['신고인부호', '특송업체부호','수입자부호','해외거래처부호'], axis=1)

In [219]:
t_df['신고일자'] = pd.to_datetime(t_df['신고일자'])
t_df.insert(1, '신고일자_월', t_df['신고일자'].dt.month)
t_df = t_df.drop('신고일자', axis=1)

In [220]:
# label encoding
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
label = t_df['수입통관계획코드']
le.fit(label)
label_encoded = le.transform(label)
l_e_df = pd.DataFrame(label_encoded, columns=['label_encoded'])
t_df['수입통관계획코드'] = l_e_df


label = t_df['수입신고구분코드']
le.fit(label)
label_encoded = le.transform(label)
l_e_df = pd.DataFrame(label_encoded, columns=['label_encoded'])
t_df['수입신고구분코드'] = l_e_df

le = LabelEncoder()
label = t_df['원산지국가코드']
le.fit(label)
label_encoded = le.transform(label)
l_e_df = pd.DataFrame(label_encoded, columns=['label_encoded'])
t_df['원산지국가코드'] = l_e_df

le = LabelEncoder()
label = t_df['적출국가코드']
le.fit(label)
label_encoded = le.transform(label)
l_e_df = pd.DataFrame(label_encoded, columns=['label_encoded'])
t_df['적출국가코드'] = l_e_df

t_df = t_df.drop(['관세율구분코드'], axis=1)

In [221]:
# 표준화 
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler() 
scaler.fit(t_df)
t_df_scaled = scaler.transform(t_df)

t_df_scaled = pd.DataFrame(data=t_df_scaled,columns=cx)

# 정규화
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() 
scaler.fit(t_df)
t_df_scaled = scaler.transform(t_df)

t_df_scaled = pd.DataFrame(data=t_df_scaled, columns=cx)

In [222]:
Xu_test = np.array(pd.DataFrame(t_df_scaled, columns = cx))
t_dt_prediction_1 = df_xgb_u.predict(Xu_test)
t_df_scaled['우범여부'] =t_dt_prediction_1

Xh_test = np.array(pd.DataFrame(t_df_scaled, columns = cu))
t_dt_prediction_2 = df_xgb_h.predict(Xh_test)

In [223]:
test_result = pd.DataFrame()
test_result['신고번호'] = t_df['신고번호']
test_result['우범여부'] = t_dt_prediction_1
test_result['핵심적발'] = t_dt_prediction_2

test_result

,신고번호,우범여부,핵심적발
0,982834,1,1
1,828961,1,0
2,522066,1,1
3,999547,1,1
4,919320,0,0
...,...,...,...
10268,857321,1,0
10269,621501,1,1
10270,998367,1,0
10271,697376,1,0


In [154]:
test_result.to_csv("C:\\Users\\82108\\result.csv", encoding='utf-8-sig',index=False, header=True)

In [166]:
test_result['핵심적발'].unique()

array([0, 1])